<h1>pip</h1>

In [9]:
# Install necessary libraries
%pip install torch torchvision opencv-python matplotlib facenet-pytorch moviepy librosa transformers mediapipe

  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl (15.5 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.0
    Uninstalling numpy-2.1.0:
      Successfully uninstalled numpy-2.1.0
  Rolling back uninstall of numpy
  Moving to c:\users\harshal shah\appdata\roaming\python\python312\scripts\f2py.exe
   from C:\Users\Harshal Shah\AppData\Local\Temp\pip-uninstall-igugvlfv\f2py.exe
  Moving to c:\users\harshal shah\appdata\roaming\python\python312\scripts\numpy-config.exe
   from C:\Users\Harshal Shah\AppData\Local\Temp\pip-uninstall-igugvlfv\numpy-config.exe
  Moving to c:\users\harshal shah\appdata\roaming\python\python312\site-packages\numpy-2.1.0-cp312-cp312-win_amd64.whl
   from C:\Users\Harshal Shah\AppData\Local\Temp\pip-uninstall-uwvkjfqq\numpy-2.1.0-cp312-cp312-win_amd64.whl
  Moving to c:\users\harshal shah\appdata\roaming\python\python312\site-packages\numpy-2.1.0.dist-info\
   fro

ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'c:\\Python312\\Scripts\\f2py.exe' -> 'c:\\Python312\\Scripts\\f2py.exe.deleteme'



<h1>Dependencies</h1>

In [10]:
import torch
import torchvision.models as models
import cv2
import numpy as np
from scipy.spatial.distance import cosine
from torchvision import transforms
import torchvision.transforms as transforms
from facenet_pytorch import MTCNN
from PIL import Image
import os
import librosa
from moviepy import VideoFileClip
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import mediapipe as mp
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


<h1>Frame Anomalies</h1>

<h4>Preprocessing</h4>

In [11]:
# Define preprocessing transformations with smaller resolution
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((112, 112)),  # Smaller input size for faster processing
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
])

# Function to preprocess a single frame
def preprocess_frame(frame):
    frame = transform(frame).unsqueeze(0)  # Add batch dimension
    return frame

<h4>Loading pre trained model</h4>

In [12]:
# Load a smaller pre-trained model (e.g., MobileNet)
model = models.mobilenet_v2(pretrained=True)
model = torch.nn.Sequential(*list(model.children())[:-1])  # Remove the final classification layer
model.eval()

# Function to extract features from a frame using the pre-trained model
def extract_features(frame, model):
    with torch.no_grad():
        features = model(frame)
    return features.squeeze().numpy().flatten()  # Flatten the feature vector to 1D

# Threshold for anomaly detection (you may need to tune this based on your data)
ANOMALY_THRESHOLD = 0.85

C:\Users\Harshal Shah\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Harshal Shah\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<h4>Detection</h4>

In [13]:
# Function to calculate cosine similarity between two feature vectors
def cosine_similarity(vec1, vec2):
    return 1 - cosine(vec1, vec2)

# Function to detect frame anomalies and display only abnormal frames
def detect_frame_anomalies(video_path, skip_frames=5):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video at path {video_path}")
        return 0, 0

    prev_features = None
    frame_count = 0
    total_frames = 0
    abnormal_frames = 0

    # Adjust the ANOMALY_THRESHOLD for better detection
    ANOMALY_THRESHOLD = 0.85  # Consider lowering this value for more sensitivity

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Skip frames to reduce processing load
        frame_count += 1
        if frame_count % skip_frames != 0:
            continue

        # Increment total frame count
        total_frames += 1

        # Preprocess the frame
        input_tensor = preprocess_frame(frame)

        # Extract features using the pre-trained model
        current_features = extract_features(input_tensor, model)

        # Compare with previous frame's features
        if prev_features is not None:
            similarity = cosine_similarity(prev_features, current_features)

            # Detect anomaly based on similarity threshold
            if similarity < ANOMALY_THRESHOLD:
                abnormal_frames += 1

                # Display the abnormal frame
                anomaly_status = "Anomaly Detected"
                color = (0, 0, 255)  # Red color for anomalies
                cv2.putText(frame, f"Status: {anomaly_status}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

                # Show the abnormal frame (optional: use cv2_imshow for Colab)
                cv2.imshow("Frame", frame)

        # Update previous features
        prev_features = current_features

    cap.release()
    return total_frames, abnormal_frames

<h4>Testing</h4>

In [15]:
# Test the function with your video file
video_path = '000471.mp4'
skip_frames = 5

total_frames, abnormal_frames = detect_frame_anomalies(video_path, skip_frames)

# Print the results
print(f"Total Frames Processed: {total_frames}")
print(f"Total Abnormal Frames Detected: {abnormal_frames}")

Total Frames Processed: 25
Total Abnormal Frames Detected: 1


In [16]:
import os
import json
# Define the path for the results.json file
results_file_path = 'results.json'

# Prepare the data to be saved
results_data = {
    "total_frames_processed": total_frames,
    "total_abnormal_frames_detected": abnormal_frames
}

try:
    # If the file exists and is not empty, read and update existing data
    if os.path.exists(results_file_path) and os.path.getsize(results_file_path) > 0:
        with open(results_file_path, 'r') as file:
            existing_data = json.load(file)
        # Update the existing data with new results
        existing_data.update(results_data)
        results_data = existing_data
    
    # Write the results to the file
    with open(results_file_path, 'w') as file:
        json.dump(results_data, file, indent=4)
    
    print(f"Results saved to {results_file_path}: {results_data}")

except Exception as e:
    print(f"Error handling results file: {str(e)}")

Results saved to results.json: {'total_frames': 25, 'distorted_faces': 0, 'total_frames_processed': 25, 'total_abnormal_frames_detected': 1, 'face_detection_rate': 97.47, 'cosine_similarity': -0.03, 'mismatch_score': 1.03, 'euclidean_distance': 1.43, 'analysis_result': 'High mismatch detected. Audio and visual content are inconsistent.'}
